In [4]:
"""
Q3: Extra Work
"""
import keras
from keras import layers
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.models import Sequential
from keras.preprocessing import sequence
import torch
import torch.nn as nn
import nltk
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import KFold

In [7]:
"""
Utility functions
"""
def flatten(matrix):
    flat_list = []
    for row in matrix:
        flat_list += row
    return flat_list

In [9]:
# set the random seed
torch.manual_seed(777)

# prepare data
all_categories = ['DET', 'NOUN', 'ADJ', 'VERB', 'ADP', '.', 'ADV', 'CONJ', 'PRT', 'PRON', 'NUM', 'X']
tagged_sentences = nltk.corpus.treebank.tagged_sents(tagset='universal')
all_sentences = []
for sentence in tagged_sentences:
    t = [word for (word, tag) in sentence]
    all_sentences.append(t)

In [11]:
# preprocess data
sequence_length = 500
max_features = 20000
embedding_dim = 128
max_sentence = 200

def custom_standardize(x):
    return x

# create vectorize layer
vectorize_layer = keras.layers.TextVectorization(
    standardize=custom_standardize,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# create vocabulary
text_ds = flatten(all_sentences)
vectorize_layer.adapt(text_ds)

# vectorize data
text_vectors = [vectorize_layer(s) for s in all_sentences]
padded_X = sequence.pad_sequences(text_vectors, maxlen=max_sentence)
X = torch.tensor(padded_X, dtype=torch.long)
print(X.shape)

all_tags = [[all_categories.index(tag) for (word, tag) in sentence] for sentence in tagged_sentences]
padded_Y = sequence.pad_sequences(all_tags, maxlen=max_sentence)
y = torch.tensor(padded_Y, dtype=torch.long)
print(y.shape)

torch.Size([3914, 200, 500])
torch.Size([3914, 200])


In [12]:
embedding = Embedding(
    input_dim=max_features,
    output_dim=64,
)
lstm = LSTM(4)
test_output = lstm(X)
print(test_output.shape)

# create the model
model = Sequential()
model.add(embedding)
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
print(model.summary())

X_train, X_test = X[:3200], X[3200:]
y_train, y_test = y[:3200], y[3200:]

# fit data
model.fit(
    X_train,
    y_train,
    epochs=3,
    batch_size=64
)

# model evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

(3914, 4)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3


ValueError: Input 0 of layer "lstm_1" is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (64, 200, 500, 64)